Zelle 1: Imports + Pfade

In [1]:
import os, json, re
import numpy as np
import pandas as pd

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

In [2]:
import sys
print(sys.executable)

c:\Users\Anwender\Documents\Repositories\BuzzwordLearner\venv312\Scripts\python.exe


In [3]:

''' Batu pahts

# Deine Struktur aus dem Screenshot
DEPT_CSV = "/Users/batuklkn/Desktop/BuzzwordLearner/data/department-v2.csv"
SEN_CSV  = "/Users/batuklkn/Desktop/BuzzwordLearner/data/seniority-v2.csv"
CV_ANN   = "/Users/batuklkn/Desktop/BuzzwordLearner/data/linkedin-cvs-annotated.json"
CV_RAW   = "/Users/batuklkn/Desktop/BuzzwordLearner/data/linkedin-cvs-not-annotated.json"

# Modellwahl:
# - wenn Titel stark deutsch/mehrsprachig sind -> multilingual
# - sonst reicht uncased
MODEL_NAME = "distilbert-base-multilingual-cased"
# MODEL_NAME = "distilbert-base-uncased"


'''

' Batu pahts\n\n# Deine Struktur aus dem Screenshot\nDEPT_CSV = "/Users/batuklkn/Desktop/BuzzwordLearner/data/department-v2.csv"\nSEN_CSV  = "/Users/batuklkn/Desktop/BuzzwordLearner/data/seniority-v2.csv"\nCV_ANN   = "/Users/batuklkn/Desktop/BuzzwordLearner/data/linkedin-cvs-annotated.json"\nCV_RAW   = "/Users/batuklkn/Desktop/BuzzwordLearner/data/linkedin-cvs-not-annotated.json"\n\n# Modellwahl:\n# - wenn Titel stark deutsch/mehrsprachig sind -> multilingual\n# - sonst reicht uncased\nMODEL_NAME = "distilbert-base-multilingual-cased"\n# MODEL_NAME = "distilbert-base-uncased"\n\n\n'

In [4]:
'''Julien paths'''

# Updated paths for Windows
DEPT_CSV = r"c:\Users\Anwender\Documents\Repositories\BuzzwordLearner\data\department-v2.csv"
SEN_CSV  = r"c:\Users\Anwender\Documents\Repositories\BuzzwordLearner\data\seniority-v2.csv"
CV_ANN   = r"c:\Users\Anwender\Documents\Repositories\BuzzwordLearner\data\linkedin-cvs-annotated.json"
CV_RAW   = r"c:\Users\Anwender\Documents\Repositories\BuzzwordLearner\data\linkedin-cvs-not-annotated.json"
# Modellwahl:
# - wenn Titel stark deutsch/mehrsprachig sind -> multilingual
# - sonst reicht uncased
MODEL_NAME = "distilbert-base-multilingual-cased"
# MODEL_NAME = "distilbert-base-uncased"

Zelle 2: CSVs laden

In [5]:
dept_df = pd.read_csv(DEPT_CSV)
sen_df  = pd.read_csv(SEN_CSV)

assert set(dept_df.columns) >= {"text", "label"}
assert set(sen_df.columns)  >= {"text", "label"}

display(dept_df.head())
display(sen_df.head())

print("Department labels:", dept_df["label"].nunique())
print("Seniority labels:", sen_df["label"].nunique())

,text,label
0,Adjoint directeur communication,Marketing
1,Advisor Strategy and Projects,Project Management
2,Beratung & Projekte,Project Management
3,Beratung & Projektmanagement,Project Management
4,Beratung und Projektmanagement kommunale Partner,Project Management


,text,label
0,Analyst,Junior
1,Analyste financier,Junior
2,Anwendungstechnischer Mitarbeiter,Junior
3,Application Engineer,Senior
4,Applications Engineer,Senior


Department labels: 11
Seniority labels: 5


Zelle 3: JSON Loader (annotated + not-annotated)

In [6]:
def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

cvs_annotated = load_json(CV_ANN)
cvs_raw = load_json(CV_RAW)

print("annotated type:", type(cvs_annotated))
print("raw type:", type(cvs_raw))

annotated type: <class 'list'>
raw type: <class 'list'>


Zelle 4: Annotated JSON laden und zu eval_df machen

In [7]:
import json
import pprint

with open(CV_ANN, "r", encoding="utf-8") as f:
    ann = json.load(f)

print("Type ann:", type(ann))
print("Type ann[0]:", type(ann[0]))
print("Type ann[0][0]:", type(ann[0][0]))

print("\nKeys of a POSITION dict:")
print(ann[0][0].keys())

print("\nFull example position:")
pprint.pprint(ann[0][0], depth=5)

Type ann: <class 'list'>
Type ann[0]: <class 'list'>
Type ann[0][0]: <class 'dict'>

Keys of a POSITION dict:
dict_keys(['organization', 'linkedin', 'position', 'startDate', 'endDate', 'status', 'department', 'seniority'])

Full example position:
{'department': 'Other',
 'endDate': None,
 'linkedin': 'https://www.linkedin.com/company/depot4design-gmbh',
 'organization': 'Depot4Design GmbH',
 'position': 'Prokurist',
 'seniority': 'Management',
 'startDate': '2019-08',
 'status': 'ACTIVE'}


In [8]:
# eval_df = nur ACTIVE Positionen + saubere Spaltennamen
# ann ist: list[list[dict]]  → erst flatten
positions = [p for cv in ann for p in cv]
eval_df = pd.DataFrame(positions)

# nur ACTIVE
eval_df["status"] = eval_df["status"].astype(str).str.upper()
eval_df = eval_df[eval_df["status"] == "ACTIVE"].copy()

# Jobtitel-Text ist bei dir "position" -> wir nennen ihn "title"
eval_df["title"] = eval_df["position"].astype(str).str.strip()

# Labels säubern
eval_df["department"] = eval_df["department"].astype(str).str.strip()
eval_df["seniority"]  = eval_df["seniority"].astype(str).str.strip()

# nur benötigte Spalten
eval_df = eval_df[["title", "department", "seniority", "organization", "status"]].copy()

display(eval_df.head(10))
print("Eval ACTIVE records:", len(eval_df))
print("Dept labels present:", eval_df["department"].notna().sum())
print("Sen labels present:", eval_df["seniority"].notna().sum())
print("Unique dept labels in eval:", eval_df["department"].nunique())
print("Unique seniority labels in eval:", eval_df["seniority"].nunique())

,title,department,seniority,organization,status
0,Prokurist,Other,Management,Depot4Design GmbH,ACTIVE
1,CFO,Other,Management,Depot4Design GmbH,ACTIVE
2,Betriebswirtin,Other,Professional,Depot4Design GmbH,ACTIVE
3,Prokuristin,Other,Management,Depot4Design GmbH,ACTIVE
4,CFO,Other,Management,Depot4Design GmbH,ACTIVE
6,Solutions Architect,Information Technology,Professional,Computer Solutions,ACTIVE
14,Medizintechnik Beratung,Consulting,Professional,Udo Weber,ACTIVE
17,Director expansión de negocio.,Business Development,Director,Grupo Viajes Kontiki.,ACTIVE
18,Gerente comercial,Sales,Lead,Air & Ground Operations Consultancy,ACTIVE
19,Administrador Unico,Administrative,Professional,Viajes Oceano S.L.,ACTIVE


Eval ACTIVE records: 623
Dept labels present: 623
Sen labels present: 623
Unique dept labels in eval: 11
Unique seniority labels in eval: 6


Zelle 4b: Not-annotated JSON als raw_df für Inference

In [9]:
# raw_df für Inference aus not-annotated JSON
# raw ist: list[list[dict]] → flatten
positions = [p for cv in cvs_raw for p in cv]
raw_df = pd.DataFrame(positions)

# falls status existiert: nur ACTIVE
if "status" in raw_df.columns:
    raw_df["status"] = raw_df["status"].astype(str).str.upper()
    raw_df = raw_df[raw_df["status"] == "ACTIVE"].copy()

# position -> title
raw_df["title"] = raw_df["position"].astype(str).str.strip()

# minimal benötigte Spalten
keep_cols = ["title"]
if "organization" in raw_df.columns:
    keep_cols.append("organization")

raw_df = raw_df[keep_cols].copy()

display(raw_df.head(10))
print("Raw records:", len(raw_df))

,title,organization
0,Bookkeeper,"Keeping The Books, Bookkeeping"
1,Co-Owner,Playful Paws
8,Strategy & Investments,Erste Bank und Sparkasse
21,Corporate Auditor,Guido Meyer
22,Corporate Auditor,Guido Meyer
24,Marketing Manager,Tradeware AG
32,Professor,Monash University
33,"Deputy Dean, Faculty of IT",Monash University
36,Research Director,Oceania Cyber Security Centre #OCSC
42,Program Purchasing Leader / Program / Acquisit...,Faurecia Innenraum Systeme GmbH


Raw records: 419


Zelle 5: Train+Eval Funktionen (generisch für Department oder Seniority)

In [10]:
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

CUDA available: True
GPU: NVIDIA GeForce RTX 5070


In [11]:
from transformers import EarlyStoppingCallback

def train_and_eval(
    train_df,
    eval_df,
    train_text_col,
    train_label_col,
    eval_text_col,
    eval_label_col,
    task_name,
    model_name=MODEL_NAME,
    max_epochs=100,
    batch_size=256,
    lr=2e-5,
    seed=42,
    patience=10,
    warmup_ratio=0.06,
    weight_decay=0.01
):
    # Eval nur dort, wo Label vorhanden ist
    eval_task = eval_df[eval_df[eval_label_col].notna()].copy()

    # Label-Encoding auf Train
    le = LabelEncoder()
    y_train = le.fit_transform(train_df[train_label_col].astype(str))
    classes = list(le.classes_)

    # Eval auf Labels beschränken, die im Train vorkommen
    eval_task = eval_task[eval_task[eval_label_col].astype(str).isin(set(classes))].copy()
    y_eval = le.transform(eval_task[eval_label_col].astype(str))

    print(f"\n=== {task_name} ===")
    print("Train:", len(train_df), "Eval:", len(eval_task))
    print("Num classes:", len(classes))
    print("LR:", lr, "max_epochs:", max_epochs, "patience:", patience)

    train_ds = Dataset.from_dict({
        "text": train_df[train_text_col].astype(str).tolist(),
        "labels": y_train.tolist()
    })
    eval_ds = Dataset.from_dict({
        "text": eval_task[eval_text_col].astype(str).tolist(),
        "labels": y_eval.tolist()
    })

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tok(batch):
        return tokenizer(batch["text"], truncation=True)

    train_ds = train_ds.map(tok, batched=True)
    eval_ds  = eval_ds.map(tok, batched=True)

    collator = DataCollatorWithPadding(tokenizer=tokenizer)

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(classes)
    )

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        return {
            "accuracy": accuracy_score(labels, preds),
            "f1_macro": f1_score(labels, preds, average="macro"),
            "f1_weighted": f1_score(labels, preds, average="weighted"),
        }

    args = TrainingArguments(
        output_dir=f"./out_{task_name}",
        eval_strategy="epoch",          # transformers >= 4.46 (GPU ok). falls error: evaluation_strategy="epoch"
        save_strategy="epoch",
        logging_strategy="steps",
        logging_steps=50,

        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        greater_is_better=True,

        learning_rate=lr,
        lr_scheduler_type="linear",
        warmup_ratio=warmup_ratio,

        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,

        num_train_epochs=max_epochs,
        weight_decay=weight_decay,

        seed=seed,
        report_to="none",

        fp16=torch.cuda.is_available(),   # auf T4 schneller
    )

    callbacks = [EarlyStoppingCallback(early_stopping_patience=patience)]

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        tokenizer=tokenizer,
        data_collator=collator,
        compute_metrics=compute_metrics,
        callbacks=callbacks
    )

    trainer.train()

    # Best model ist bereits geladen (load_best_model_at_end=True)
    metrics = trainer.evaluate()
    print("Best-model metrics:", metrics)

    # Predictions auf Eval
    pred = trainer.predict(eval_ds)
    pred_ids = np.argmax(pred.predictions, axis=-1)
    pred_labels = le.inverse_transform(pred_ids)

    pred_df = eval_task[[eval_text_col, eval_label_col]].copy()
    pred_df["pred"] = pred_labels
    pred_df["correct"] = pred_df[eval_label_col].astype(str) == pred_df["pred"].astype(str)

    print("\nClassification report (best model):")
    print(classification_report(
        y_true=y_eval,
        y_pred=pred_ids,
        target_names=le.classes_,
        digits=4
    ))

    return trainer, le, metrics, pred_df

In [12]:
lrs = [5e-5, 3e-5, 2e-5]
results = []

for lr in lrs:
    _, _, m, _ = train_and_eval(
        train_df=dept_df,
        eval_df=eval_df,
        train_text_col="text",
        train_label_col="label",
        eval_text_col="title",
        eval_label_col="department",
        task_name=f"department_lr{lr}",
        max_epochs=100,
        batch_size=256,
        lr=lr,
        patience=10
    )
    results.append((lr, m["eval_f1_macro"]))

print(results)
best_lr = max(results, key=lambda x: x[1])[0]
print("Best LR:", best_lr)


=== department_lr5e-05 ===
Train: 10145 Eval: 623
Num classes: 11
LR: 5e-05 max_epochs: 100 patience: 10


Map:   0%|          | 0/10145 [00:00<?, ? examples/s]

Map:   0%|          | 0/623 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Anwender\AppData\Local\Temp\ipykernel_26228\1094497307.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,No log,2.574832,0.035313,0.006202,0.002409
2,2.137000,2.857149,0.160514,0.079607,0.067386
3,0.801200,2.416984,0.195827,0.175079,0.121408
4,0.189100,2.492372,0.210273,0.217029,0.152437
5,0.079900,2.435669,0.243981,0.308636,0.214801
6,0.079900,2.817081,0.250401,0.306766,0.194089
7,0.038000,2.659141,0.277689,0.348772,0.261995
8,0.017400,3.115707,0.234350,0.335266,0.210496
9,0.009600,3.381778,0.261637,0.321959,0.194171
10,0.008200,3.381455,0.261637,0.304181,0.201964


...

Map:   0%|          | 0/10145 [00:00<?, ? examples/s]

Map:   0%|          | 0/623 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Anwender\AppData\Local\Temp\ipykernel_26228\1094497307.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,No log,2.384092,0.043339,0.015552,0.009870
2,2.262300,2.849577,0.086677,0.031276,0.022020
3,1.455700,2.699015,0.174960,0.112743,0.079459
4,0.484100,2.370149,0.242376,0.200611,0.144683
5,0.162700,2.167212,0.253612,0.229350,0.177440
6,0.162700,2.279896,0.285714,0.283259,0.245061
7,0.084200,2.345826,0.290530,0.331026,0.238386
8,0.042900,2.668856,0.277689,0.351359,0.198927
9,0.025400,2.701287,0.293740,0.368944,0.236323
10,0.015500,2.595790,0.314607,0.374470,0.283790


Best-model metrics: {'eval_loss': 2.6842474937438965, 'eval_accuracy': 0.32102728731942215, 'eval_f1_macro': 0.3795378591562141, 'eval_f1_weighted': 0.2800545383429739, 'eval_runtime': 0.0606, 'eval_samples_per_second': 10285.984, 'eval_steps_per_second': 49.531, 'epoch': 21.0}

Classification report (best model):
                        precision    recall  f1-score   support

        Administrative     0.0396    0.2857    0.0696        14
  Business Development     0.2727    0.3000    0.2857        20
            Consulting     0.2500    0.5897    0.3511        39
      Customer Support     1.0000    0.3333    0.5000         6
       Human Resources     0.3077    0.5000    0.3810        16
Information Technology     0.3007    0.7419    0.4279        62
             Marketing     0.3750    0.5455    0.4444        22
                 Other     0.8485    0.0814    0.1485       344
    Project Management     0.3333    0.8462    0.4783        39
            Purchasing     0.2917    0.4667

In [13]:
print("dept_df:", dept_df.shape, "labels:", dept_df["label"].nunique())
print("sen_df :", sen_df.shape,  "labels:", sen_df["label"].nunique())
print("eval_df:", eval_df.shape)
print("raw_df :", raw_df.shape)

print("\nEval dept counts:")
print(eval_df["department"].value_counts().head(15))
print("\nEval seniority counts:")
print(eval_df["seniority"].value_counts().head(15))

dept_df: (10145, 2) labels: 11
sen_df : (9428, 2) labels: 5
eval_df: (623, 5)
raw_df : (419, 2)

Eval dept counts:
department
Other                     344
Information Technology     62
Sales                      46
Consulting                 39
Project Management         39
Marketing                  22
Business Development       20
Human Resources            16
Purchasing                 15
Administrative             14
Customer Support            6
Name: count, dtype: int64

Eval seniority counts:
seniority
Professional    216
Management      192
Lead            125
Senior           44
Director         34
Junior           12
Name: count, dtype: int64


In [14]:
dept_trainer, dept_le, dept_metrics, dept_pred_df = train_and_eval(
    train_df=dept_df,
    eval_df=eval_df,
    train_text_col="text",
    train_label_col="label",
    eval_text_col="title",
    eval_label_col="department",
    task_name="department",
    max_epochs=30,
    batch_size=256,
    lr=2e-5,
    patience=10
)

display(dept_pred_df.head(20))
print("Dept eval correct mean:", dept_pred_df["correct"].mean())


=== department ===
Train: 10145 Eval: 623
Num classes: 11
LR: 2e-05 max_epochs: 30 patience: 10


Map:   0%|          | 0/10145 [00:00<?, ? examples/s]

Map:   0%|          | 0/623 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Anwender\AppData\Local\Temp\ipykernel_26228\1094497307.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,No log,2.659085,0.040128,0.013291,0.008017
2,2.007100,2.703327,0.176565,0.114053,0.079905
3,0.563000,2.212604,0.239165,0.221201,0.153158
4,0.134000,2.354487,0.247191,0.261831,0.175662
5,0.071800,2.516413,0.269663,0.307837,0.195152
6,0.071800,2.324592,0.290530,0.349486,0.253485
7,0.041300,2.661734,0.277689,0.380493,0.224221
8,0.022700,2.796736,0.276083,0.366677,0.217100
9,0.016800,2.892839,0.276083,0.379141,0.229926
10,0.012500,2.817678,0.279294,0.376514,0.227683


Best-model metrics: {'eval_loss': 2.6617343425750732, 'eval_accuracy': 0.27768860353130015, 'eval_f1_macro': 0.38049262416111773, 'eval_f1_weighted': 0.22422072398248352, 'eval_runtime': 0.0592, 'eval_samples_per_second': 10515.636, 'eval_steps_per_second': 50.637, 'epoch': 17.0}

Classification report (best model):
                        precision    recall  f1-score   support

        Administrative     0.0292    0.2857    0.0530        14
  Business Development     0.2727    0.3000    0.2857        20
            Consulting     0.4884    0.5385    0.5122        39
      Customer Support     0.4000    0.3333    0.3636         6
       Human Resources     0.8000    0.5000    0.6154        16
Information Technology     0.2419    0.7258    0.3629        62
             Marketing     0.3235    0.5000    0.3929        22
                 Other     0.8000    0.0233    0.0452       344
    Project Management     0.2500    0.7949    0.3804        39
            Purchasing     0.5455    0.40

,title,department,pred,correct
0,Prokurist,Other,Project Management,False
1,CFO,Other,Information Technology,False
2,Betriebswirtin,Other,Information Technology,False
3,Prokuristin,Other,Project Management,False
4,CFO,Other,Information Technology,False
6,Solutions Architect,Information Technology,Information Technology,True
14,Medizintechnik Beratung,Consulting,Marketing,False
17,Director expansión de negocio.,Business Development,Information Technology,False
18,Gerente comercial,Sales,Marketing,False
19,Administrador Unico,Administrative,Information Technology,False


Dept eval correct mean: 0.27768860353130015


In [15]:
sen_trainer, sen_le, sen_metrics, sen_pred_df = train_and_eval(
    train_df=sen_df,
    eval_df=eval_df,
    train_text_col="text",
    train_label_col="label",
    eval_text_col="title",
    eval_label_col="seniority",
    task_name="seniority",
    max_epochs=30,
    batch_size=256,
    lr=2e-5,
    patience=10
)

display(sen_pred_df.head(20))
print("Sen eval correct mean:", sen_pred_df["correct"].mean())


=== seniority ===
Train: 9428 Eval: 407
Num classes: 5
LR: 2e-05 max_epochs: 30 patience: 10


Map:   0%|          | 0/9428 [00:00<?, ? examples/s]

Map:   0%|          | 0/407 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Anwender\AppData\Local\Temp\ipykernel_26228\1094497307.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,No log,1.639595,0.272727,0.151715,0.175055
2,1.407000,1.296937,0.528256,0.447608,0.513756
3,0.361400,1.247014,0.697789,0.635517,0.717994
4,0.361400,1.344278,0.705160,0.623020,0.725096
5,0.060500,1.435422,0.707617,0.635705,0.725927
6,0.020200,1.488459,0.705160,0.617991,0.724176
7,0.015900,1.515415,0.722359,0.632829,0.740657
8,0.015900,1.583318,0.707617,0.618055,0.728898
9,0.008600,1.680974,0.707617,0.627840,0.725464
10,0.006100,1.673563,0.707617,0.608387,0.730042


Best-model metrics: {'eval_loss': 1.4354218244552612, 'eval_accuracy': 0.7076167076167076, 'eval_f1_macro': 0.6357054709995886, 'eval_f1_weighted': 0.7259269852852741, 'eval_runtime': 0.0397, 'eval_samples_per_second': 10245.298, 'eval_steps_per_second': 50.345, 'epoch': 15.0}

Classification report (best model):
              precision    recall  f1-score   support

    Director     0.5741    0.9118    0.7045        34
      Junior     0.4167    0.4167    0.4167        12
        Lead     0.8333    0.6400    0.7240       125
  Management     0.9362    0.6875    0.7928       192
      Senior     0.3846    0.9091    0.5405        44

    accuracy                         0.7076       407
   macro avg     0.6290    0.7130    0.6357       407
weighted avg     0.7994    0.7076    0.7259       407



,title,seniority,pred,correct
0,Prokurist,Management,Junior,False
1,CFO,Management,Management,True
3,Prokuristin,Management,Junior,False
4,CFO,Management,Management,True
17,Director expansión de negocio.,Director,Director,True
18,Gerente comercial,Lead,Junior,False
24,"APL-ansvarig, samordning",Lead,Lead,True
32,Kaufmännischer Leiter,Lead,Lead,True
34,Lab-Supervisor,Lead,Senior,False
42,Managing Director,Management,Director,False


Sen eval correct mean: 0.7076167076167076


In [16]:
from datasets import Dataset
import numpy as np

def predict_on_raw(trainer, label_encoder, raw_df, text_col="title", out_path="predictions.csv"):
    ds = Dataset.from_dict({"text": raw_df[text_col].astype(str).tolist()})
    tokenizer = trainer.tokenizer

    def tok(batch):
        return tokenizer(batch["text"], truncation=True)
    ds = ds.map(tok, batched=True)

    pred = trainer.predict(ds)
    pred_ids = np.argmax(pred.predictions, axis=-1)
    pred_labels = label_encoder.inverse_transform(pred_ids)

    out = raw_df.copy()
    out["pred"] = pred_labels
    out.to_csv(out_path, index=False)
    print("Saved:", out_path)
    return out

In [17]:
dept_raw_pred = predict_on_raw(dept_trainer, dept_le, raw_df, out_path="raw_pred_department.csv")
sen_raw_pred  = predict_on_raw(sen_trainer,  sen_le, raw_df, out_path="raw_pred_seniority.csv")

display(dept_raw_pred.head(10))
display(sen_raw_pred.head(10))

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Map:   0%|          | 0/419 [00:00<?, ? examples/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Saved: raw_pred_department.csv


Map:   0%|          | 0/419 [00:00<?, ? examples/s]

Saved: raw_pred_seniority.csv


,title,organization,pred
0,Bookkeeper,"Keeping The Books, Bookkeeping",Information Technology
1,Co-Owner,Playful Paws,Administrative
8,Strategy & Investments,Erste Bank und Sparkasse,Information Technology
21,Corporate Auditor,Guido Meyer,Marketing
22,Corporate Auditor,Guido Meyer,Marketing
24,Marketing Manager,Tradeware AG,Marketing
32,Professor,Monash University,Administrative
33,"Deputy Dean, Faculty of IT",Monash University,Information Technology
36,Research Director,Oceania Cyber Security Centre #OCSC,Consulting
42,Program Purchasing Leader / Program / Acquisit...,Faurecia Innenraum Systeme GmbH,Purchasing


,title,organization,pred
0,Bookkeeper,"Keeping The Books, Bookkeeping",Senior
1,Co-Owner,Playful Paws,Management
8,Strategy & Investments,Erste Bank und Sparkasse,Management
21,Corporate Auditor,Guido Meyer,Senior
22,Corporate Auditor,Guido Meyer,Senior
24,Marketing Manager,Tradeware AG,Senior
32,Professor,Monash University,Junior
33,"Deputy Dean, Faculty of IT",Monash University,Director
36,Research Director,Oceania Cyber Security Centre #OCSC,Director
42,Program Purchasing Leader / Program / Acquisit...,Faurecia Innenraum Systeme GmbH,Senior


In [18]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

y_true = dept_pred_df["department"].astype(str)
y_pred = dept_pred_df["pred"].astype(str)

print("=== Department (Eval) ===")
print("Accuracy :", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred, average="macro"))
print("Recall   :", recall_score(y_true, y_pred, average="macro"))
print("F1       :", f1_score(y_true, y_pred, average="macro"))

=== Department (Eval) ===
Accuracy : 0.27768860353130015
Precision: 0.44611940437348435
Recall   : 0.46139602376596406
F1       : 0.38049262416111773


In [19]:
y_true = sen_pred_df["seniority"].astype(str)
y_pred = sen_pred_df["pred"].astype(str)

print("=== Seniority (Eval) ===")
print("Accuracy :", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred, average="macro"))
print("Recall   :", recall_score(y_true, y_pred, average="macro"))
print("F1       :", f1_score(y_true, y_pred, average="macro"))

=== Seniority (Eval) ===
Accuracy : 0.7076167076167076
Precision: 0.6289719342910832
Recall   : 0.7130044563279857
F1       : 0.6357054709995886
